In [2]:
import pandas as pd
from sklearn.metrics import mean_squared_error  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr.models import WDL #wide&deep
from deepctr.inputs import SparseFeat,get_feature_names

data = pd.read_csv('movielens_sample.txt')
print(data.head())
sparse_features = ["movie_id","user_id","gender","age","occupation","zip"]
target = ["rating"]

   user_id  movie_id  rating  timestamp  \
0     3299       235       4  968035345   
1     3630      3256       3  966536874   
2      517       105       4  976203603   
3      785      2115       3  975430389   
4     5848       909       5  957782527   

                                         title            genres gender  age  \
0                               Ed Wood (1994)      Comedy|Drama      F   25   
1                         Patriot Games (1992)   Action|Thriller      M   18   
2        Bridges of Madison County, The (1995)     Drama|Romance      F   25   
3  Indiana Jones and the Temple of Doom (1984)  Action|Adventure      M   18   
4                        Apartment, The (1960)      Comedy|Drama      M   50   

   occupation    zip  
0           4  19119  
1           4  77005  
2          14  55408  
3          19  29307  
4          20  20009  


In [6]:
#对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
#计算每个特征中不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature,data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns+dnn_feature_columns)

#将数据集切分成训练集和测试集
train,test = train_test_split(data,test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

#使用WDL进行训练
model = WDL(linear_feature_columns,dnn_feature_columns,task='regression')
model.compile('adam','mse',metrics=['mse'],)
history = model.fit(train_model_input,train[target].values,batch_size=256,epochs=1,verbose=True,validation_split=0.2,)
#使用WDL进行预测
pred_ans = model.predict(test_model_input,batch_size=256)
#输出RMSE或MSE
mse = round(mean_squared_error(test[target].values,pred_ans),4)
rmse = mse**0.5
print('test RMSE',rmse)

Train on 128 samples, validate on 32 samples
128/128 [==============================] - 2s 16ms/sample - loss: 13.9143 - mean_squared_error: 13.9143 - val_loss: 15.6719 - val_mean_squared_error: 15.6719
test RMSE 3.6674377977001873
